In [1]:
%reset
%matplotlib inline
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import scipy.stats
import mne
from mne.time_frequency import tfr_morlet
from mne.stats import permutation_cluster_1samp_test
import gc
import os
import copy
from scipy.signal import firwin, lfilter, find_peaks
import pickle
from os.path import exists
import mne
import numpy as np
from mne import create_info
from IPython.utils import io
import yasa
import seaborn as sns
import pandas as pd
import matlab
import matlab.engine
import os
import shutil
from scipy.signal import butter, lfilter
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from scipy.signal import freqz
from scipy import signal

def butter_bandpass(lowcut, highcut, fs, order=5):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def overlap(a, b):
    return a[1] >= b[0] and a[0] <= b[1]

def restart_output_dir(path):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.mkdir(path)

gc.collect()

37830

In [2]:
pkl_dir = "C:\\Users\\User\\Cloud-Drive\\BigFiles\\OmissionExpOutput\\eventDetection\\try2\\imported_eventDetectionChan\\no_filters"
sub = '33'
sleep_data_file_num = '2'


import_type = "eventDetectionChan"
event_detection_dir_name = 'eventDetection'
electrode_column_name = 'electrode'
header_edf = np.array(['Onset',"Duration","Annotation"])
spindles_output_columns =  ['spindleStartTime', 'spindleEndTime', 'peakTime', 'peakEnergy', 'peakEnergyNorm', 'freqSpindle', 'spindleDuration/SR', 'PowerSP', 'PowerAlpha', 'sleepStage']
kcomp_df_columns = ['pos_i','pos_amp','neg_i','neg_amp','power',electrode_column_name]

In [33]:
def edfViewFormat_scoring_dict(score):
    if score == 0:
        return 'W'
    elif score ==1:
        return 'N1'
    elif score ==2:
        return 'N2'
    elif score ==3:
        return 'N3'
    elif score ==4:
        return 'TREM'
    elif score ==5:
        return 'PREM'
    elif score ==6:
        return 'MOVE'
    elif score ==7:
        return 'ARTIFACT'
    else:
        Exception('no such score')

def get_edfViewFormat_scoring(scoring):
    new_format_score = np.zeros((len(scoring),3), dtype=object)
    for ind, score in enumerate(scoring):
            new_format_score[ind,:] = [30*ind,30,edfViewFormat_scoring_dict(score)] ## onset (sec), duration, desc
    return new_format_score

def group_events(filterd_events_allElectrodes, start_column_name, end_column_name,power_column_name,electrode_column_name):
    # The grouping is done such that you get 1 event per time frame.
    # In the single condition, only one electrode is sufficiant to include event.
    # In the multi condition event will be includede only if it appears in sevral electrodes
    # The picked electrode is the one where the event is with the most power
    filterd_events_allElectrodes.sort_values(by=[start_column_name],inplace=True) 
    filterd_events_allElectrodes.reset_index(drop=True, inplace=True)
    deleted = filterd_events_allElectrodes.copy(deep=True)
    filtered = pd.DataFrame([], columns = deleted.columns)
    filtered_moreThan1 = pd.DataFrame([], columns = deleted.columns)
    simultan = pd.DataFrame([], columns = deleted.columns)

    while len(deleted)>0:
        if len(simultan)==0:
            simultan = pd.concat([simultan, deleted.iloc[[0]]])
            deleted.drop(deleted.index[0], axis=0, inplace=True)
            still_overlap = True
        else:
            while still_overlap == True and len(deleted)>0:
                still_overlap = False
                simultan.reset_index(drop=True, inplace=True) # make sure indexes pair with number of rows

                ## check now_overlap_all_in_simultan:
                for index, simultan_row in simultan.iterrows():
                    simultan_0 = simultan_row[start_column_name]
                    simultan_1 = simultan_row[end_column_name]
                    deleted_0 = deleted.iloc[0][start_column_name]
                    deleted_1 = deleted.iloc[0][end_column_name]
                    if overlap([simultan_0,simultan_1],[deleted_0,deleted_1]):
                        simultan = pd.concat([simultan, deleted.iloc[[0]]])
                        deleted.drop(deleted.index[0], axis=0, inplace=True)
                        still_overlap = True
                        break
                if still_overlap: continue
                else:
                    ## check max ps and add to filt
                    # if len(simultan)>1:
                    #     print('hu')
                    simultan.sort_values(by=[power_column_name],ascending=False,inplace=True)
                    row_df = simultan.iloc[[0]]
                    row_df.iat[0, row_df.columns.get_loc(electrode_column_name)] = np.unique(np.array(simultan[electrode_column_name]))
                    filtered = pd.concat([filtered,row_df])
                    if len(simultan) > 1: filtered_moreThan1 = pd.concat([filtered_moreThan1,row_df])
                    simultan = pd.DataFrame([], columns = deleted.columns)                    
        
    filtered.reset_index(drop=True, inplace=True) # make sure indexes pair with number of rows
    filtered_moreThan1.reset_index(drop=True, inplace=True) # make sure indexes pair with number of rows
    return filtered, filtered_moreThan1
    # allsubsdata_perFile[id][uniqeElctd_ss_key] = filtered   
    # allsubsdata_perFile[id][uniqeElctds_ss_key] = filtered_moreThan1   
    # print(f"sub:{id}, before filt:{np.shape(allsubsdata_perFile[id][ss_key_to_use])[0]}, after:{np.shape(allsubsdata_perFile[id][uniqeElctd_ss_key])[0]}, after>1:{np.shape(allsubsdata_perFile[id][uniqeElctds_ss_key])[0]}")

def preform_event_comparison(all_event_manu,all_event_auto,file_id,event_tag):

    ## get the array of before manual scanning
    all_event_auto = np.delete(all_event_auto, np.where(all_event_auto[:, 2] == "Annotation")[0], axis=0)
    all_event_auto[:,[0,1]] = [np.double(x) for x in all_event_auto[:,[0,1]]]
    event_ind = np.array([],dtype=int)
    for ind_i, desc in enumerate(all_event_auto[:,2]):
        if event_tag in desc:
            event_ind = np.append(event_ind, ind_i)
    all_event_auto = all_event_auto[event_ind,:]
    all_event_auto[:, 1] = np.where(all_event_auto[:, 1] < 0.1, all_event_auto[:, 1] * configu['sample_freq'], all_event_auto[:, 1])# solves the division by sample rate issue

    ## get the array of after manual scanning
    all_event_manu = np.delete(all_event_manu, np.where(all_event_manu[:, 2] == "Annotation")[0], axis=0)
    all_event_manu[:,[0,1]] = [np.double(x) for x in all_event_manu[:,[0,1]]]
    event_ind = np.array([],dtype=int)
    for ind_i, desc in enumerate(all_event_manu[:,2]):
        if event_tag == 'kc':
            if event_tag in desc or "Kcomp" in desc:
                event_ind = np.append(event_ind, ind_i)
        else:
            if event_tag in desc:
                event_ind = np.append(event_ind, ind_i)
    all_event_manu = all_event_manu[event_ind,:]
    all_event_manu[:, 1] = np.where(all_event_manu[:, 1] < 0.1, all_event_manu[:, 1] * configu['sample_freq'], all_event_manu[:, 1])# solves the division by sample rate issue


    ## compare to find rate of TP and FP
    TP = 0
    FP = 0
    for ss_auto in all_event_auto:
        found = False
        for ss_manu in all_event_manu:
            if overlap([ss_auto[0],ss_auto[0]+ss_auto[1]],[ss_manu[0],ss_manu[0]+ss_manu[1]]):
                TP +=1
                found = True
                break
        if found == False:
            FP +=1

    # FN = 0
    # for ss_manu in all_ss_manu:
    #     found = False
    #     for ss_auto in all_ss_auto:
    #         if overlap([ss_auto[0],ss_auto[0]+ss_auto[1]],[ss_manu[0],ss_manu[0]+ss_manu[1]]):
    #             found = True
    #             break
    #     if found == False:
    #         FN +=1

    Positive = np.shape(all_event_manu)[0]   
    hitRate = TP/Positive
    #missRate = FN/Positive
    falseDiscoveryRate = FP/(FP+TP)
    
    comparison = [file_id, Positive, TP, FP,hitRate, falseDiscoveryRate]
    return comparison

def preform_ss_comparison_yasa(manu_detection_filepath,sub,sleep_data_file_num,yasa_ss_summary):
    file_id = f'{sub}_{sleep_data_file_num}'
    EEG_data, configu, datafile_scoring, exmp_scoring_upsampled = get_eeg_data(pkl_dir,sub,sleep_data_file_num)

    indeces_ss_auto = np.zeros(np.shape(EEG_data)[1]);
    auto_startTime_inSamples_arr = np.array(yasa_ss_summary['Start']* configu['sample_freq'],dtype=np.int64) 
    auto_endTime_inSamples_arr = np.array(yasa_ss_summary['End']* configu['sample_freq'],dtype=np.int64) 
    for ss_start,ss_end in zip(auto_startTime_inSamples_arr,auto_endTime_inSamples_arr):
        indeces_ss_auto[ss_start:ss_end] = 1;

    ## get the array of after manual scanning
    all_ss_manu = np.loadtxt(manual_detection_filepath, delimiter="\t",dtype='object')
    all_ss_manu = np.delete(all_ss_manu, np.where(all_ss_manu[:, 2] == "Annotation")[0], axis=0)
    all_ss_manu[:,[0,1]] = [np.double(x) for x in all_ss_manu[:,[0,1]]]
    ss_ind = np.array([],dtype=int)
    for ind_i, desc in enumerate(all_ss_manu[:,2]):
        if ("SS" in desc) or ("ss" in desc):
            ss_ind = np.append(ss_ind, ind_i)
    all_ss_manu = all_ss_manu[ss_ind,:]
    all_ss_manu[:, 1] = np.where(all_ss_manu[:, 1] < 0.1, all_ss_manu[:, 1] * configu['sample_freq'], all_ss_manu[:, 1])# solves the division by sample rate issue

    indeces_ss_manu = np.zeros(np.shape(EEG_data)[1]);
    manu_startTime_inSamples_arr = np.array(all_ss_manu[:,0]* configu['sample_freq'],dtype=np.int64)
    manu_duration_inSamples_arr = np.array(all_ss_manu[:,1]* configu['sample_freq'],dtype=np.int64) 
    for ss_start,ss_dur in zip(manu_startTime_inSamples_arr,manu_duration_inSamples_arr):
        indeces_ss_manu[ss_start : ss_start+ss_dur] = 1;

    return yasa.compare_detection(indices_detection=np.squeeze(np.nonzero(indeces_ss_auto)),
                                  indices_groundtruth=np.squeeze(np.nonzero(indeces_ss_manu)),
                                   max_distance=int(configu['sample_freq']))

# returns False if file doesnt exists. If it does, return: data, configu, datafile_scoring, exmp_scoring_upsampled
def get_eeg_data(pkl_dir,sub,sleep_data_file_num):
    import_path = f'{pkl_dir}\\{import_type}_sub-{sub}_sleep-{sleep_data_file_num}.pkl'
    if os.path.isfile(import_path):
        with open(import_path, "rb") as file:  [allsubsdata_perFile, configu] = pickle.load(file)
        id = f"{sub}_{sleep_data_file_num}"
        EEG_data = allsubsdata_perFile[id]['data']; ## shape (electrode, time)
        datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)
        exmp_scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, EEG_data, sf_data=configu['sample_freq'])
        return EEG_data, configu, datafile_scoring, exmp_scoring_upsampled
    else:
        return False

def kc_detection(eeg_timeseries, sample_rate,included_sleepstages, sleepstages_upsampled):
    # Subtract mean from x
    eeg_timeseries = eeg_timeseries - np.mean(eeg_timeseries)

    # Bandpass filter parameters
    a = firwin(100, [0.25/sample_rate,5/sample_rate], pass_zero='bandpass')
    filtered_eeg = lfilter(a, [1], eeg_timeseries)

    # Invert the data to find troughs instead of peaks (actually troughs in original signal)
    minimas_indexes, minimas_popr = find_peaks(np.max(filtered_eeg) - filtered_eeg, distance=1.5*sample_rate,width=0.15*sample_rate)
    minimas = filtered_eeg[minimas_indexes]
    minimas_width = minimas_popr['widths']

    # Thresholding
    mm = minimas[minimas <= -45]
    pos = np.where(np.isin(minimas, mm))[0]
    sig_minimas_indxs = minimas_indexes[pos]
    minimas_width = minimas_width[pos]

    # Further filtering of peaks and computing durations
    durations = []
    kc_onset = []
    kc_end = []
    kc_min_pos= []
    kc_min_power= []
    kc_max_pos= []
    kc_max_power= []
    kc_peak_difference = [];

    for sig_minima_samp_i, sig_minima_width in zip(sig_minimas_indxs, minimas_width):
        baseline_before_minima = np.mean(filtered_eeg[sig_minima_samp_i-int(1.5*sample_rate) : sig_minima_samp_i - int(sample_rate*0.5)]) # mean 1-2 sec before it
        if filtered_eeg[sig_minima_samp_i] < 2.5* baseline_before_minima and \
        sleepstages_upsampled[sig_minima_samp_i] in included_sleepstages :
            # Find the subsequent positive peak after the detected negative peak (within 1 second after the negative peak)
            maximas_samp_relative_i, maximas_popr = find_peaks(filtered_eeg[sig_minima_samp_i:], distance=sample_rate, width=0.15*sample_rate) 
            
            if maximas_samp_relative_i.size > 0:
                maxima_samp_i = sig_minima_samp_i+maximas_samp_relative_i[0]
                maxima_val = filtered_eeg[maxima_samp_i]
                maxima_width = maximas_popr['widths'][0] 
                kc_duration = ( maxima_samp_i - (sig_minima_samp_i - (sig_minima_width/2))) / sample_rate # (end of kc - start of kc) / samplerate
                negative_peak = filtered_eeg[sig_minima_samp_i]
                positive_peak = filtered_eeg[sig_minima_samp_i + maximas_samp_relative_i[0]]
                peak_difference = positive_peak - negative_peak

                # Filtering conditions: 
                if maxima_val > 20 and \
                    kc_duration >= 0.5 and kc_duration <= 2.5 and \
                    sig_minima_width < maxima_width and \
                    baseline_before_minima < maxima_val and \
                    peak_difference >= 100 :
                    # sig_minima_width / configu['sample_freq'] < negative_peak / 170 and 
                    durations.append(kc_duration)
                    kc_onset.append(sig_minima_samp_i - (sig_minima_width/2))
                    kc_min_pos.append(sig_minima_samp_i)
                    kc_min_power.append(negative_peak)
                    kc_max_power.append(positive_peak)
                    kc_max_pos.append(maxima_samp_i)
                    kc_end.append(sig_minima_samp_i - (sig_minima_width/2) + (kc_duration*sample_rate))
                    kc_peak_difference.append(peak_difference)

    res = {
        "durations": durations,
        "kc_onset": kc_onset,
        "kc_min_pos": kc_min_pos,
        "kc_min_power": kc_min_power,
        "kc_max_power": kc_max_power,
        "kc_max_pos": kc_max_pos,
        "kc_peak_difference": kc_peak_difference,
        "kc_end": kc_end
    }

    return res 

def preform_kc_comparison_yasa(manu_detection_filepath,sub,sleep_data_file_num,auto_detection_filepath):
    EEG_data, configu, datafile_scoring, exmp_scoring_upsampled = get_eeg_data(pkl_dir,sub,sleep_data_file_num)

    ## get the array of after manual scanning
    all_kc_auto = np.loadtxt(auto_detection_filepath, delimiter="\t",dtype='object')
    all_kc_auto = np.delete(all_kc_auto, np.where(all_kc_auto[:, 2] == "Annotation")[0], axis=0)
    all_kc_auto[:,[0,1]] = [np.double(x) for x in all_kc_auto[:,[0,1]]]
    ss_ind = np.array([],dtype=int)
    for ind_i, desc in enumerate(all_kc_auto[:,2]):
        if ("kc" in desc) or ("Kcomp" in desc):
            ss_ind = np.append(ss_ind, ind_i)
    all_kc_auto = all_kc_auto[ss_ind,:]
    all_kc_auto[:, 1] = np.where(all_kc_auto[:, 1] < 0.1, all_kc_auto[:, 1] * configu['sample_freq'], all_kc_auto[:, 1])# solves the division by sample rate issue

    indeces_kc_auto = np.zeros(np.shape(EEG_data)[1]);
    auto_startTime_inSamples_arr = np.array(all_kc_auto[:,0]* configu['sample_freq'],dtype=np.int64)
    auto_duration_inSamples_arr = np.array(all_kc_auto[:,1]* configu['sample_freq'],dtype=np.int64) 
    for kc_start,kc_dur in zip(auto_startTime_inSamples_arr,auto_duration_inSamples_arr):
        indeces_kc_auto[kc_start : kc_start+kc_dur] = 1;


    ## get the array of after manual scanning
    all_kc_manu = np.loadtxt(manu_detection_filepath, delimiter="\t",dtype='object')
    all_kc_manu = np.delete(all_kc_manu, np.where(all_kc_manu[:, 2] == "Annotation")[0], axis=0)
    all_kc_manu[:,[0,1]] = [np.double(x) for x in all_kc_manu[:,[0,1]]]
    ss_ind = np.array([],dtype=int)
    for ind_i, desc in enumerate(all_kc_manu[:,2]):
        if ("kc" in desc) or ("Kcomp" in desc):
            ss_ind = np.append(ss_ind, ind_i)
    all_kc_manu = all_kc_manu[ss_ind,:]
    all_kc_manu[:, 1] = np.where(all_kc_manu[:, 1] < 0.1, all_kc_manu[:, 1] * configu['sample_freq'], all_kc_manu[:, 1])# solves the division by sample rate issue

    indeces_kc_manu = np.zeros(np.shape(EEG_data)[1]);
    manu_startTime_inSamples_arr = np.array(all_kc_manu[:,0]* configu['sample_freq'],dtype=np.int64)
    manu_duration_inSamples_arr = np.array(all_kc_manu[:,1]* configu['sample_freq'],dtype=np.int64) 
    for kc_start,kc_dur in zip(manu_startTime_inSamples_arr,manu_duration_inSamples_arr):
        indeces_kc_manu[kc_start : kc_start+kc_dur] = 1;

    return yasa.compare_detection(indices_detection=np.squeeze(np.nonzero(indeces_kc_auto)),
                                  indices_groundtruth=np.squeeze(np.nonzero(indeces_kc_manu)),
                                   max_distance=int(configu['sample_freq']))


In [ ]:
# save only scoring. one file
sub = '33'
sleep_data_file_num = '2'

id = f"{sub}_{sleep_data_file_num}"
EEG_data, configu, datafile_scoring, exmp_scoring_upsampled = get_eeg_data(pkl_dir,sub,sleep_data_file_num)
all_events_with_header = np.vstack([header_edf,get_edfViewFormat_scoring(datafile_scoring)])
np.savetxt(f"{pkl_dir}\\scoring_{id}.txt", all_events_with_header, delimiter='\t',fmt='%s')

In [4]:
# comparison Spindles
subs = ["32","33",'35',"38"]

comparisons_df = pd.DataFrame(columns=['id','f1','recall','precision', 'tp','fp','fn'])
for sub in subs:
    EEG_data, configu, datafile_scoring, exmp_scoring_upsampled = get_eeg_data(pkl_dir,sub,sleep_data_file_num)

    res_kc = yasa.spindles_detect(EEG_data,sf=configu['sample_freq'], hypno = exmp_scoring_upsampled,
                            verbose=False,
                            freq_sp=(11, 16),
                            thresh={"rel_pow": 0.2, "corr": 0.65, "rms": 1.5},
                            include=[2, 3],ch_names=configu['electrodes_names'],
                            multi_only=False,
                            remove_outliers = False)
    
    manual_detection_filepath = f"{pkl_dir}\\SS-YASA+Sleepstage_{sub}_{sleep_data_file_num}_MANUAL.txt"
    if os.path.exists(manual_detection_filepath):
        comp_res = preform_ss_comparison_yasa(manual_detection_filepath,sub,sleep_data_file_num,res_kc.summary())
        comparison = [f"{sub}_{sleep_data_file_num}",comp_res['f1'], comp_res['recall'], comp_res['precision'], np.size(comp_res['tp']), np.size(comp_res['fp']),np.size(comp_res['fn'])]
        comparisons_df.loc[len(comparisons_df)] = comparison

display(comparisons_df)
# all_comp.to_csv('ss_comparisons.csv',index=False)

23-ספט-23 19:51:11 | WARNING | Hypnogram is LONGER than data by 3.59 seconds. Cropping hypnogram to match data.size.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n

,id,f1,recall,precision,tp,fp,fn
0,32_2,0.732361,0.870534,0.632042,53436,31109,7947
1,33_2,0.872374,0.822370,0.928853,343616,26320,74220
2,35_2,0.546022,0.545644,0.546402,14949,12410,12448
3,38_2,0.912288,0.941741,0.884620,354156,46192,21909


In [31]:
# comparison K-complex
subs = ["33","38"]

comparisons_df = pd.DataFrame(columns=['id','f1','recall','precision', 'tp','fp','fn','Positive', 'hitRate', 'falseDiscoveryRate'])
for sub in subs:
    file_id = f"{sub}_{sleep_data_file_num}"
    EEG_data, configu, datafile_scoring, exmp_scoring_upsampled = get_eeg_data(pkl_dir,sub,sleep_data_file_num)
    auto_kc_edfForm = np.vstack([header_edf,get_edfViewFormat_scoring(datafile_scoring)])
    kcomp_electrodes = configu['electrodes_names'][~np.isin(configu['electrodes_names'], ['O1', 'O2'])]
    auto_kc_edf_filepath = f"{pkl_dir}\\KC+Sleepstage_{sub}_{sleep_data_file_num}.txt"
    if not os.path.exists(auto_kc_edf_filepath):
        for elec_i, elecd in enumerate(kcomp_electrodes):
            res_kc = kc_detection(EEG_data[elec_i],configu['sample_freq'],[2],exmp_scoring_upsampled)
            pos_kc = np.array(res_kc['kc_onset']) / configu['sample_freq']

            startTime_arr = np.array(pos_kc)
            durations_arr = np.array(res_kc['durations'])
            desc = [f"kc@@{elecd}" for x in startTime_arr]
            auto_kc_edfForm_curr_electrode = np.array([startTime_arr ,durations_arr,desc],dtype=object).T ## onset (sec), duration,desc
            auto_kc_edfForm = np.vstack([auto_kc_edfForm,auto_kc_edfForm_curr_electrode])
        np.savetxt(auto_kc_edf_filepath, auto_kc_edfForm, delimiter='\t',fmt='%s')

    manual_detection_filepath = f"{pkl_dir}\\KC+Sleepstage_{file_id}_MANUAL.txt"
    if os.path.exists(manual_detection_filepath):
        all_kc_manu = np.loadtxt(manual_detection_filepath, delimiter="\t",dtype='object')

        comp_res = preform_kc_comparison_yasa(manual_detection_filepath,sub,sleep_data_file_num,auto_kc_edf_filepath)
        comparison = [f"{sub}_{sleep_data_file_num}",comp_res['f1'], comp_res['recall'], comp_res['precision'] \
                      , np.size(comp_res['tp']), np.size(comp_res['fp']),np.size(comp_res['fn'])    \
                      , np.shape(all_kc_manu)[0] 
                      , np.size(comp_res['tp']) / np.shape(all_kc_manu)[0] \
                      , np.size(comp_res['fp']) / (np.size(comp_res['tp'])+np.size(comp_res['fp'])) \
                      ]
        comparisons_df.loc[len(comparisons_df)] = comparison
        
display(comparisons_df)
# all_comp.to_csv('kc_comparisons.csv',index=False)

23-ספט-23 22:22:01 | WARNING | Hypnogram is LONGER than data by 27.44 seconds. Cropping hypnogram to match data.size.


KeyboardInterrupt: 

## Run and save detection for all subject 

In [50]:
# SS  - run and save
subs = ["30","33",'35',"38"]
for sub in subs:
    ss_df_curr_sub = pd.DataFrame();
    for filenum in range(1,6):
        file_id = f"{sub}_{filenum}"
        if not get_eeg_data(pkl_dir,sub,filenum): continue
        EEG_data, configu, datafile_scoring, exmp_scoring_upsampled = get_eeg_data(pkl_dir,sub,filenum)
        auto_ss_edfForm = np.vstack([header_edf,get_edfViewFormat_scoring(datafile_scoring)])
        auto_ss_edf_filepath = f"{pkl_dir}\\SS-YASA+Sleepstage_{file_id}.txt"
        auto_detection_ss_filepath_csv =f'{pkl_dir}//sub-{sub}_Spindles.csv'

        # create csv - spindles
        res_ss = yasa.spindles_detect(EEG_data,sf=configu['sample_freq'], hypno = exmp_scoring_upsampled,
            verbose=False,
            freq_sp=(11, 16),
            thresh={"rel_pow": 0.2, "corr": 0.65, "rms": 1.5},
            include=[2, 3],ch_names=configu['electrodes_names'],
            multi_only=False,
            remove_outliers = False)
        ss_summary = res_ss.summary()
        
        filtered, filtered_moreThan1 = group_events(res_ss.summary(), 'Start','End','RelPower',"Channel")
        
        # create edf event 
        startTime_arr = np.array(ss_summary['Start'])
        endTime_arr = np.array(ss_summary['End'])
        duration_arr = endTime_arr - startTime_arr
        electd_arr_per_ss = np.array(ss_summary['Channel'])
        desc = [f"ss-YASA@@{electd_arr}" for electd_arr in electd_arr_per_ss]
        auto_ss_edfForm_curr_electrode = np.array([startTime_arr ,durations_arr,desc],dtype=object).T ## onset (sec), duration,desc
        auto_ss_edfForm = np.vstack([auto_ss_edfForm,auto_ss_edfForm_curr_electrode])
        np.savetxt(auto_ss_edf_filepath, auto_ss_edfForm, delimiter='\t',fmt='%s')


        new_df = pd.DataFrame(ss_summary)
        new_df['sub'] = sub
        new_df['subfile'] = filenum
        new_df['eventType'] = 'Spindles'
        if ss_df_curr_sub.empty:
            ss_df_curr_sub= new_df
        else:   
            ss_df_curr_sub = pd.concat([ss_df_curr_sub,new_df])

    # save csv
    if(ss_df_curr_sub.size>0):
        ss_df_curr_sub.to_csv(auto_detection_ss_filepath_csv)


23-ספט-23 23:51:04 | WARNING | Hypnogram is LONGER than data by 0.46 seconds. Cropping hypnogram to match data.size.
23-ספט-23 23:51:04 | WARNING | Hypnogram is LONGER than data by 0.46 seconds. Cropping hypnogram to match data.size.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  

In [39]:
# KC  - run and save
subs = ["30","33",'35',"38"]
for sub in subs:
    kc_df_curr_sub = pd.DataFrame();
    for filenum in range(1,6):
        file_id = f"{sub}_{filenum}"
        if not get_eeg_data(pkl_dir,sub,filenum): continue;
        EEG_data, configu, datafile_scoring, exmp_scoring_upsampled = get_eeg_data(pkl_dir,sub,filenum)
        auto_kc_edfForm = np.vstack([header_edf,get_edfViewFormat_scoring(datafile_scoring)])
        kcomp_electrodes = configu['electrodes_names'][~np.isin(configu['electrodes_names'], ['O1', 'O2'])]
        auto_kc_edf_filepath = f"{pkl_dir}\\KC+Sleepstage_{sub}_{filenum}.txt"
        auto_detection_kc_filepath_csv =f'{pkl_dir}//sub-{sub}_KComp.csv'
        if not os.path.exists(auto_kc_edf_filepath) or not os.path.exists(auto_detection_kc_filepath_csv):
            for elec_i, elecd in enumerate(kcomp_electrodes):
                res_kc = kc_detection(EEG_data[elec_i],configu['sample_freq'],[2],exmp_scoring_upsampled)
                pos_kc = np.array(res_kc['kc_onset']) / configu['sample_freq']

                # create edf event 
                startTime_arr = np.array(pos_kc)
                durations_arr = np.array(res_kc['durations'])
                desc = [f"kc@@{elecd}" for x in startTime_arr]
                auto_kc_edfForm_curr_electrode = np.array([startTime_arr ,durations_arr,desc],dtype=object).T ## onset (sec), duration,desc
                auto_kc_edfForm = np.vstack([auto_kc_edfForm,auto_kc_edfForm_curr_electrode])
                
                # create csv - kcomp
                new_df = pd.DataFrame(res_kc)
                new_df['sub'] = sub
                new_df['subfile'] = filenum
                new_df['eventType'] = 'KComp'
                new_df['electrode'] = elecd
                if kc_df_curr_sub.empty:
                    kc_df_curr_sub= new_df
                else:   
                    kc_df_curr_sub = pd.concat([kc_df_curr_sub,new_df])

            # save csv
            if(kc_df_curr_sub.size>0):
                filtered, filtered_moreThan1 = group_events(kc_df_curr_sub, "kc_onset", "kc_end",'kc_peak_difference','electrode')
                filtered.to_csv(auto_detection_kc_filepath_csv)

            # save edf
            np.savetxt(auto_kc_edf_filepath, auto_kc_edfForm, delimiter='\t',fmt='%s')
        

23-ספט-23 22:52:54 | WARNING | Hypnogram is LONGER than data by 0.46 seconds. Cropping hypnogram to match data.size.
23-ספט-23 22:52:54 | WARNING | Hypnogram is LONGER than data by 0.46 seconds. Cropping hypnogram to match data.size.
23-ספט-23 22:54:26 | WARNING | Hypnogram is LONGER than data by 3.38 seconds. Cropping hypnogram to match data.size.
23-ספט-23 22:54:26 | WARNING | Hypnogram is LONGER than data by 3.38 seconds. Cropping hypnogram to match data.size.
c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  ret

## ARCHIVE

In [ ]:
# sw detection
sw = yasa.sw_detect(datafile_data,sf=configu['sample_freq'], hypno = exmp_scoring_upsampled,
                          include=[2, 3],ch_names=configu['electrodes_names'])
sw.summary()

In [ ]:
# comparison Spindles - old detection params
subs = ["32","33",'35',"38"]

comparisons_df = pd.DataFrame(columns=['id','Positive','TruePos','FalsePos', 'hitRate','falseDiscoveryRate'])
for sub in subs:
    EEG_data, configu, datafile_scoring, exmp_scoring_upsampled = get_eeg_data(pkl_dir,sub,sleep_data_file_num)

    res_kc = yasa.spindles_detect(EEG_data,sf=configu['sample_freq'], hypno = exmp_scoring_upsampled,
                            verbose=False,
                            freq_sp=(11, 16),
                            thresh={"rel_pow": 0.2, "corr": 0.65, "rms": 1.5},
                            include=[2, 3],ch_names=configu['electrodes_names'],
                            multi_only=True,
                            remove_outliers = True)
    # print(sp)
    # print(sp.summary())
    # sp.plot_average(center='Peak',time_before = 1,time_after = 1)
    # plt.show()
    #%matplotlib widget
    #sp.plot_detection()

    sp_summary = res_kc.summary()
    file_id = f"{sub}_{sleep_data_file_num}"

    startTime_arr = np.array(sp_summary['Start'])
    endTime_arr = np.array(sp_summary['End'])
    duration_arr = endTime_arr - startTime_arr
    electd_arr_per_ss = np.array(sp_summary['Channel'])
    desc = [f"ss-YASA@@{electd_arr}" for electd_arr in electd_arr_per_ss]
    auto_ss_edfForm = np.array([startTime_arr ,duration_arr,desc],dtype=object).T ## onset (sec), duration,desc

    manual_detection_filepath = f"{pkl_dir}\\SS-YASA+Sleepstage_{file_id}_MANUAL.txt"
    if os.path.exists(manual_detection_filepath):
        all_event_manu = np.loadtxt(manual_detection_filepath, delimiter="\t",dtype='object')

        comparison = preform_event_comparison(all_event_manu,auto_ss_edfForm,file_id)
        comparisons_df.loc[len(comparisons_df)] = comparison
        
display(comparisons_df)
# all_comp.to_csv('ss_comparisons.csv',index=False)